<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/igbobert45_Test_Code_14_12_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
#!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt
!wget -c https://raw.githubusercontent.com/chiamaka249/IgboNER/main/for_test.txt

--2021-12-14 21:35:14--  https://raw.githubusercontent.com/chiamaka249/IgboNER/main/for_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10225 (10.0K) [text/plain]
Saving to: ‘for_test.txt’

for_test.txt        100%[===================>]   9.99K  --.-KB/s    in 0s      

2021-12-14 21:35:14 (46.0 MB/s) - ‘for_test.txt’ saved [10225/10225]



In [2]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers==2.11.0
!pip install tokenizers==0.8.0rc1
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 674 kB 5.6 MB/s 
     |████████████████████████████████| 5.6 MB 32.2 MB/s 
     |████████████████████████████████| 1.2 MB 29.9 MB/s 
     |████████████████████████████████| 895 kB 35.6 MB/s 
     |████████████████████████████████| 3.0 MB 5.4 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 2.11.0 requires tokenizers==0.7.0, but you have tokenizers 0.8.0rc1 which is incompatible.
tokenizers                    0.8.0rc1
transformers                  2.11.0


In [3]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]
print(paths)

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=2_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

['for_test.txt']
CPU times: user 39.4 ms, sys: 17.1 ms, total: 56.6 ms
Wall time: 41.2 ms


In [4]:
!mkdir igbo_bert45
tokenizer.save_model("igbo_bert45")

['igbo_bert45/vocab.json', 'igbo_bert45/merges.txt']

In [5]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert45/vocab.json",
    "./igbo_bert45/merges.txt",
)

In [6]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [7]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.").tokens

['<s>',
 'S',
 'i',
 'm',
 'on',
 'e',
 'Ġgara',
 'Ġá»¥ka',
 'Ġá»¥',
 'ny',
 'ahá»¥',
 'Ġg',
 'u',
 'á»į',
 'Ġe',
 'gw',
 'u',
 'Ġma',
 'Ġga',
 '-',
 'kwa',
 'Ġ',
 'ta',
 'a',
 '.',
 '</s>']

In [8]:
# Check that we have a GPU
!nvidia-smi

Tue Dec 14 21:41:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [9]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=2_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [10]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./igbo_bert45", max_len=512,config=config)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [11]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [12]:
model.num_parameters()
# => 83 million parameters

45645008

In [13]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/for_test.txt",
    block_size = 128
)

CPU times: user 12 ms, sys: 608 µs, total: 12.6 ms
Wall time: 17.4 ms


In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./igbo_bert45",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2    
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [17]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 2.22 s, sys: 59.1 ms, total: 2.28 s
Wall time: 2.55 s


TrainOutput(global_step=3, training_loss=7.394643306732178)

In [18]:
trainer.save_model("./igbo_bert45")

In [19]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert45",
    tokenizer="./igbo_bert45"
)

In [20]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ")

[{'score': 0.014455199241638184,
  'sequence': '<s> Nwaanyị na na</s>',
  'token': 267},
 {'score': 0.005965663585811853,
  'sequence': "<s> Nwaanyị na'</s>",
  'token': 11},
 {'score': 0.003932390362024307,
  'sequence': '<s> Nwaanyị na n</s>',
  'token': 262},
 {'score': 0.0022346917539834976,
  'sequence': '<s> Nwaanyị na.</s>',
  'token': 18},
 {'score': 0.0021408211905509233,
  'sequence': '<s> Nwaanyị nachi</s>',
  'token': 403}]

In [21]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.012935477308928967,
  'sequence': '<s> Nwaanyị na na ji na akara.</s>',
  'token': 267},
 {'score': 0.0059821465983986855,
  'sequence': "<s> Nwaanyị na' ji na akara.</s>",
  'token': 11},
 {'score': 0.004087827168405056,
  'sequence': '<s> Nwaanyị na n ji na akara.</s>',
  'token': 262},
 {'score': 0.0022835161071270704,
  'sequence': '<s> Nwaanyị na. ji na akara.</s>',
  'token': 18},
 {'score': 0.0021366216242313385,
  'sequence': '<s> Nwaanyị na e ji na akara.</s>',
  'token': 293}]

In [22]:
# The sun <mask>.
# =>

fill_mask("Chineke ga-ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.017372598871588707,
  'sequence': '<s> Chineke ga-ebibikwa ndị niile na- eme ihe na.</s>',
  'token': 267},
 {'score': 0.005170090589672327,
  'sequence': "<s> Chineke ga-ebibikwa ndị niile na- eme ihe'.</s>",
  'token': 11},
 {'score': 0.0037967669777572155,
  'sequence': '<s> Chineke ga-ebibikwa ndị niile na- eme ihe n.</s>',
  'token': 262},
 {'score': 0.0027717389166355133,
  'sequence': '<s> Chineke ga-ebibikwa ndị niile na- eme ihe nke.</s>',
  'token': 289},
 {'score': 0.0026602966245263815,
  'sequence': '<s> Chineke ga-ebibikwa ndị niile na- eme ihechi.</s>',
  'token': 403}]

In [23]:
fill_mask("O riri <mask>.")

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.02062103897333145,
  'sequence': '<s> O riri na.</s>',
  'token': 267},
 {'score': 0.007785310503095388, 'sequence': "<s> O riri'.</s>", 'token': 11},
 {'score': 0.00383514235727489, 'sequence': '<s> O riri n.</s>', 'token': 262},
 {'score': 0.003393922233954072,
  'sequence': '<s> O ririchi.</s>',
  'token': 403},
 {'score': 0.0022363048046827316, 'sequence': '<s> O riri,.</s>', 'token': 16}]

In [25]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.024361122399568558,
  'sequence': '<s> Ọ bụ naa ka a na- arịa.</s>',
  'token': 267},
 {'score': 0.004521827679127455,
  'sequence': "<s> Ọ bụ'a ka a na- arịa.</s>",
  'token': 11},
 {'score': 0.0033757216297090054,
  'sequence': '<s> Ọ bụ na ka a na- arịa.</s>',
  'token': 262},
 {'score': 0.0026853843592107296,
  'sequence': '<s> Ọ bụ.a ka a na- arịa.</s>',
  'token': 18},
 {'score': 0.0024172740522772074,
  'sequence': '<s> Ọ bụ kọa ka a na- arịa.</s>',
  'token': 737}]

In [26]:
import shutil

In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [28]:
shutil.move('/content/igbo_bert45','/content/gdrive/MyDrive/iFOLDER')

'/content/gdrive/MyDrive/iFOLDER/igbo_bert45'

In [29]:
shutil.move('/content/runs','/content/gdrive/MyDrive/iFOLDER')

'/content/gdrive/MyDrive/iFOLDER/runs'